# ERT Data Assimilation
**Notebook to reproduce outputs from Mary et al 2023**

Authors: Benjamin Mary, ORCID: 0000-0001-7199-2885

---

This is a notebook to reproduce outputs from  [Mary et al (2023)](link to come) 

:::{important}
# Cite this work
We believe in a community-driven approach of open-source tools that are
composable and extensible. If you use this notebook cite the work as:
> 
:::

While in the full article different scenarios are considered, for simplicity here we only show the application for the scenario 1 and 2 describe below.

The notebooks describes: 

1. **Preprocessing step**: build a mesh mimicking a rhizotron experiment.

2. **Prepare for Data Assimilation**
   - 2.1 Read observations
   - 2.1.2 Create matrice covariances
   - 2.2 Perturbate
   - 2.3 Define mapping operator
   
3. **Simulation**: solve the surface-subsurface flow.


4. **Plot outputs**: analysis of the results
   - Saturation with uncertainties
   - Assimilation quality


:::{admonition} What you should already know
In order to complete this tutorial, you should be relatively familiar with using the pyCATHY for:
- Building a mesh from a DEM (See {doc}`../content/SSHydro/index` for more information.)
- Basics of Data Assimilation
- ...
:::


```{warning}
Found a bug 🐛/ a typo ? [Email me](mailto:benjamin.mary@unipd.it)
```

We start by importing the required packages: 
- `matplotlib` in order to plot the data;
- `cathy_tools` is the main object controlling the simulation;
- `pyCATHY.DA.cathy_DA` allows you to create a Data Assimilation Object (with all CATHY tools modules inheritence);
- `perturbate` module to add perturbation to the model parameters;
- `perturbate_parm` function to perturbate a given parameter.
- `pyCATHY.DA.observations` module to read, prepare observations for DA analysis.

In [1]:
import os
import matplotlib.pyplot as plt
from pyCATHY.DA.cathy_DA import DA
from pyCATHY.DA import perturbate
from pyCATHY.DA.cathy_DA import perturbate_parm, dictObs_2pd
from pyCATHY.DA.observations import read_observations, prepare_observations, make_data_cov

In [2]:
#%% Init CATHY DA object 
# -----------------------
simu_DA = DA(dirName='.', 
            prj_name='Maryetal_2023_WRR',
            notebook=True) 

🏁 Initiate CATHY object

## 1. Preprocessing

Build an adequate mesh

## 2. Prepare Data Assimilation

First we define a dictionnary `scenario` describing which and how of the model parameter are **perturbated** and **updated**. Two cases are considered: 
- `uniroot_ic`
- `uniroot_ic_PCREF_ZROOT_updPCREF_ZROOT`


:::{admonition} Feddes parameters perturbation
In order to perturbate Feddes parameters we use typical range of possible values for **PCREF** as describe in ??. As for **ZROOT** the perturbation must be bounded within the limit of the simulation region (rhizotron domain). This condition is applied thanks to the dictionnary key `per_bounds`.
:::


- `per_type`
- `per_name`
- `per_nom`
- `per_sigma`
- `sampling_type`



In [18]:
scenario = {
    
            # scenario without parameter update
            # -------------------------------------
            'uniroot_ic': {'per_type': [None],
                                'per_name':['ic'],
                                'per_nom':[-5],
                                'per_mean':[-5],
                                'per_sigma': [1.75],
                                'transf_type':[None],
                                'listUpdateParm': ['St. var.'],
                                },
    
            # scenario with parameter update
            # -------------------------------------
            'uniroot_ic_PCREF_ZROOT_updPCREF_ZROOT': 
                                                        {'per_type': [None,None,'additive'], # this is a list of the same size than ['ic', 'ZROOT','PCREF']
                                                         'per_name':['ic', 'ZROOT','PCREF'],
                                                         'per_nom':[-5,0.4,-4],
                                                         'per_mean':[-5,0.4,-1],    
                                                         'per_sigma': [1.75,5e-3,1.55],
                                                         'per_bounds': [None,{'min':0,'max':0.5},None],
                                                         'sampling_type': ['normal']*3,
                                                         'transf_type':[None,None,None],
                                                         'ref_scenario': 'model_uni_root_ArchiePert0_f1_noise5', 
                                                         'listUpdateParm': ['St. var.', 'ZROOT','PCREF']
                                                         },    
            }
print(scenario)

{'uniroot_ic': {'per_type': [None], 'per_name': ['ic'], 'per_nom': [-5], 'per_mean': [-5], 'per_sigma': [1.75], 'transf_type': [None], 'listUpdateParm': ['St. var.']}, 'uniroot_ic_PCREF_ZROOT_updPCREF_ZROOT': {'per_type': [None, None, 'additive'], 'per_name': ['ic', 'ZROOT', 'PCREF'], 'per_nom': [-5, 0.4, -4], 'per_mean': [-5, 0.4, -1], 'per_sigma': [1.75, 0.005, 1.55], 'per_bounds': [None, {'min': 0, 'max': 0.5}, None], 'sampling_type': ['normal', 'normal', 'normal'], 'transf_type': [None, None, None], 'ref_scenario': 'model_uni_root_ArchiePert0_f1_noise5', 'listUpdateParm': ['St. var.', 'ZROOT', 'PCREF']}}


### 2.1 Import ERT observations

Read the csv files produced by pygimli

```{tip}
    need to call `read_observations` as many times as variable to perturbate 
    return a dict merging all variable perturbate to parse into prepare_DA
```

In [19]:
# ERT observations metadata
# -------------------------
ERT    = {
            'data_type': '$ERT$', # units
            'units': '$\Ohm$', # units transfer_resistances
            'forward_mesh_vtk_file': meshERT, # path to the ERT mesh (vtk file compatible with pygimli or resipy)
            'sequenceERT': sequenceERT, # path to the ERT sequence  (file compatible with pygimli or resipy)
            'instrument': 'Syscal', # Instrument
            'data_format': data_format, # format (raw or preprocessed)
        }

# Initiate an empty dictionnary
# -----------------------------
data_measure = {}
for i, tt in enumerate(time_sampled):

    # csv file observation generated by pygimli
    filename = os.path.join(pathERT, prjERT, 'ER_predicted_sol_t' + str(i) + '.csv')
    data_measure = read_observations(
                                    data_measure,
                                    filename, 
                                    data_type = 'ERT', 
                                    data_err = args.dataErr, # instrumental error
                                    show=True,
                                    tA=tt,
                                    obs_cov_type='data_err', #data_err
                                    elecs=elecs,
                                    meta=ERT
                                    ) # data_err  reciprocal_err

NameError: name 'meshERT' is not defined

#### 2.1.1 Create observation covariance matrice 

```{tip}
    For the ERT case we used a diagonal matrice
```

In [ ]:
data_measure_df = dictObs_2pd(data_measure) 
data_cov_stacked_times = [] # need to define covariance matrice for each assimilation times
for i, tt in enumerate(time_sampled):
    data_cov = data_measure_df['data_cov'].iloc[i]
    data_cov_stacked_times.append(data_cov)

simu_DA.stacked_data_cov = data_cov_stacked_times

### 2.2 Perturbate 

```{tip}
    need to call `read_observations` as many times as variable to perturbate 
    return a dict merging all variable perturbate to parse into prepare_DA
```


In [17]:
help(perturbate.perturbate)
NENS = 3
list_pert = perturbate.perturbate(simu_DA, scenario['uniroot_ic'], NENS)
print(list_pert)

Help on function perturbate in module pyCATHY.DA.perturbate:

perturbate(simu_DA, scenario, NENS)
    Write a list of dictionaries, each containing all the informations on how to
    perturbate the parameters based on the scenario to consider

[{'type_parm': 'ic', 'nominal': -5, 'mean': -5, 'sd': 1.75, 'units': 'pressure head $(m)$', 'sampling_type': 'normal', 'ensemble_size': 3, 'per_type': None, 'savefig': 'ic.png'}]


In [ ]:


var_per_dict_stacked = {}
for dp in list_pert:

    np.random.seed(1)

    # need to call perturbate_var as many times as variable to perturbate
    # return a dict merging all variable perturbate to parse into prepare_DA
    var_per_dict_stacked = perturbate_parm(
                                var_per_dict_stacked,
                                parm=dp, 
                                type_parm = dp['type_parm'], # can also be VAN GENUCHTEN PARAMETERS
                                mean =  dp['mean'],
                                sd =  dp['sd'],
                                sampling_type =  dp['sampling_type'],
                                ensemble_size =  dp['ensemble_size'], # size of the ensemble
                                per_type= dp['per_type'],
                                savefig= os.path.join(prj_name,
                                                      prj_name + dp['savefig'])
                                )

### 2.3 Define mapping operator 

Use `set_Archie_parm` with the `simu_DA` object



```{tip} Need another mapper?
    It is possible to quickly add a mapper to the DA. 
```



In [ ]:
simu_DA.set_Archie_parm(
                            rFluid_Archie=solution_parm['rFluid_Archie'],
                            a_Archie=solution_parm['a_Archie'],
                            m_Archie=solution_parm['m_Archie'],
                            n_Archie=solution_parm['n_Archie'],
                            pert_sigma_Archie=solution_parm['pert_sigma_Archie']
                        )

### 3. Run sequential DA

Simply use `run_DA_sequential()` with the `simu_DA` object



Required arguments are:
- **dict_obs**: dictionnary of observations
- **list_assimilated_obs**: list of observation to assimilate 
- **list_parm2update**: list of parameters to update 

Possible **optionnal** arguments are: 
- **parallel**: if True use multiple cores to run many realisations at the same time
- **DA_type**: type of data assimilation
- **threshold_rejected**: threshold above which the simulation stops (i.e. ensemble of rejected realisation too big)
- **damping**: add damping to inflate the covariance matrice


```{tip}
    During the execution **useful informations are displayed on the console** in order to follow the state of the DA. You can for example appreciated how many ensemble are rejected.
```



In [ ]:
simu_DA.run_DA_sequential(
                              DTMIN=1e2,
                              DELTAT=1e2,
                              parallel=parallel,    
                              dict_obs= data_measure,
                              list_assimilated_obs='all', # default
                              list_parm2update=scenarii[list(scenarii)[Snb]]['listUpdateParm'],
                              DA_type=DA_type, #'pf_analysis', # default
                              dict_parm_pert=var_per_dict_stacked,
                              open_loop_run=open_loop_run,
                              threshold_rejected=80,
                              damping=args.damping                    
                            )

### 4. Plot outputs

In [ ]:
results = simu.load_pickle_backup()
